In [0]:
CREATE OR REFRESH STREAMING TABLE bronzee_laptimes
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
       _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/ishwariformulabox/schemalanding/volume2/lap_times.csv',
  'csv',
  map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_laptimes (
  CONSTRAINT valid_id EXPECT (raceId IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_driver_id EXPECT (driverId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_lap EXPECT (lap IS NOT NULL AND lap > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_position EXPECT (position IS NOT NULL AND position > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_time EXPECT (time IS NOT NULL AND time RLIKE '^[0-9]{2}:[0-5][0-9]\\.[0-9]$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_milliseconds EXPECT (milliseconds IS NOT NULL AND milliseconds >= 0) ON VIOLATION DROP ROW
)
COMMENT 'Cleaned lap times data'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT raceId,
       driverId,
       lap,
       position,
       time,
       milliseconds
FROM STREAM(bronzee_laptimes);